In [1]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from pymetr.oscilloscope.core import Oscilloscope
from utilities import si_str_to_float

This code can be used to set the logging level of the script.

In [2]:
# Set up basic configuration for logging
logging.basicConfig(level=logging.CRITICAL, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

Set up the connection to the instrument

In [3]:
# Replace with your oscilloscope's VISA resource string
resource_string = 'TCPIP0::192.168.1.111::hislip0::INSTR'
osc = Oscilloscope(resource_string)
osc.open()
print(f"Connected to: {osc.identity()}")

Connected to: KEYSIGHT TECHNOLOGIES,DSOX1204G,CN63147376,02.12.2021071625



We configure the instrument by modifying attributes 

In [4]:
osc.acquire.count = 10000
osc.acquire.type = osc.acquire.Type.NORMAL

osc.trigger.mode = osc.trigger.Mode.EDGE  # settings which require a 
osc.trigger.source = osc.Source.CH1
osc.trigger.level = 0.5  # Volts
osc.trigger.slope = osc.trigger.Slope.POSITIVE

osc.timebase.range = 0.001

From here we can use the oscilloscope root functions to handle the data acquisition flow

In [5]:
osc.autoscale(osc.Source.CH1)
osc.single()
osc.query_operation_complete()
osc.digitize('CH1')
osc.query_operation_complete()
preamble = osc.waveform.fetch_preamble()
voltage_data = osc.waveform.fetch_trace(1)

Lastly, we can built the x-axis data and plot the result.

In [6]:
num_points = len(voltage_data)
time_data = np.arange(num_points) * osc.waveform.x_increment + osc.waveform.x_origin  #use the updated preamble data to build the x ax-s

plt.figure(figsize=(10, 6))
plt.plot(time_data, voltage_data, label='Channel 1')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('Oscilloscope Waveform')
plt.legend()
plt.grid(True)
plt.show()
osc.close()

TypeError: object of type 'NoneType' has no len()